In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("asia10K.csv", sep=None, engine='python', na_values='?')
# df = pd.read_csv("abalone.data", sep=None, engine='python', na_values='?')

index_constant = np.where(df.nunique() == 1)[0]
constant_columns = [df.columns[i] for i in index_constant]
df = df.drop(columns=constant_columns)
df = df.dropna()
cat_data = df.select_dtypes('object').astype('category')
for c in cat_data:
    df = df.assign(**{c: cat_data[c]})
float_data = df.select_dtypes('number').astype('float64')
for c in float_data:
    df = df.assign(**{c: float_data[c]})


In [3]:
import pybnesian as pbn

In [4]:
float_data.std()

Series([], dtype: float64)

In [5]:
df.head(3)

,Smoker,LungCancer,VisitToAsia,Tuberculosis,TuberculosisOrCancer,X-ray,Bronchitis,Dyspnea
0,yes,yes,no,no,yes,yes,yes,yes
1,no,no,no,no,no,yes,yes,yes
2,no,no,no,no,no,no,yes,yes


In [6]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaled_df = df.copy()
# scaled_df[float_data.columns] = scaler.fit_transform(float_data)

In [7]:
mskcmi = pbn.MSKMutualInformation(df=df, k=5, samples=20, seed=42)

In [8]:
mskcmi.mi('Smoker', 'LungCancer')

0.024363864768286038

In [9]:
# import timeit
# timeit.timeit("mskcmi.mi('Length', 'Sex')", globals=globals(), number=30)/30


In [19]:
import timeit
timeit.timeit("mskcmi.mi('Smoker', 'LungCancer', ['TuberculosisOrCancer', 'Bronchitis', 'Dyspnea', 'Tuberculosis'])", globals=globals(), number=50)/50


0.046785216999996916

In [ ]:
import timeit
timeit.timeit("mskcmi.pvalue('Smoker', 'LungCancer', ['TuberculosisOrCancer', 'Bronchitis', 'Dyspnea', 'Tuberculosis'])", globals=globals(), number=10)/200


0.1848915017750005

In [13]:
mskcmi.pvalue('Tuberculosis', 'LungCancer', 'TuberculosisOrCancer')

1.0

In [22]:
mskcmi.mi('Tuberculosis', 'LungCancer', 'TuberculosisOrCancer')

0.02319525873089496

In [9]:
mskcmi.mi('Smoker', 'VisitToAsia')

-0.00015335642231718136

In [29]:
mskcmi.mi('Smoker', 'LungCancer')

0.024363864768286038

In [8]:
mskcmi.pvalue('Tuberculosis', 'LungCancer', 'TuberculosisOrCancer')

ValueError: Column index Tuberculosis do not exist in DataFrame.

In [12]:
mskcmi.mi('Smoker', 'VisitToAsia', ['TuberculosisOrCancer'])

-0.0004076251267193244

In [130]:
mskcmi.mi('Length', 'Sex', 'Shucked weight')

-0.04036570845862181

In [171]:
mskcmi.mi('Length', 'Diameter', ['Height','Whole weight','Shucked weight'])

0.2816153296847718

In [14]:
mskcmi = pbn.MSKMutualInformation(df=df, k=5, samples=20, tree_leafsize=16)

In [31]:
import timeit
timeit.timeit("mskcmi.mi('Length', 'Sex', 'Shucked weight')", globals=globals(), number=30)/30


0.049761940466729965

In [32]:
import timeit
timeit.timeit("mskcmi.mi('Smoker', 'VisitToAsia')", globals=globals(), number=30)/30


ValueError: Column index Smoker do not exist in DataFrame.

In [ ]:
import timeit
timeit.timeit("mskcmi.mi('Smoker', 'LungCancer')", globals=globals(), number=30)/30


0.08824051839995567

In [33]:
import timeit
timeit.timeit("mskcmi.mi('Length', 'Sex')", globals=globals(), number=30)/30


0.004968380633363268

In [34]:
import timeit
timeit.timeit("mskcmi.mi('Length', 'Sex', ['Height','Whole weight','Shucked weight'])", globals=globals(), number=20)/20


0.0698433584499071

In [35]:
import timeit
timeit.timeit("mskcmi.mi('Smoker', 'LungCancer', ['TuberculosisOrCancer', 'Bronchitis', 'Dyspnea', 'Tuberculosis'])", globals=globals(), number=20)/20


ValueError: Column index Smoker do not exist in DataFrame.

In [11]:
import knncmi as ms

ms.cmi( ['Smoker'],['VisitToAsia'], ['TuberculosisOrCancer', 'Bronchitis', 'Dyspnea', 'Tuberculosis'], 30, df)

np.float64(6.30440940406908e-05)

In [15]:
ms.cmi( ['Length'], ['Sex'],['Height','Whole weight','Shucked weight'], 30, scaled_df)

np.float64(0.01409907770234893)

In [19]:
import knncmi as ms

ms.cmi( ['Length'],['Sex'], [], 30, scaled_df)

NameError: name 'scaled_df' is not defined

In [30]:
import timeit
timeit.timeit("ms.cmi( ['Smoker'], ['LungCancer'],[], 5, df)", globals=globals(), number=20)/20


3.451859858700118

In [ ]:
import timeit
timeit.timeit("ms.cmi( ['Length'], ['Diameter'],[], 30, scaled_df)", globals=globals(), number=20)/20


0.5322824361500352

In [12]:
kcmi = pbn.KMutualInformation(df=scaled_df[float_data.columns], k=5, samples=20)

In [13]:
import timeit
timeit.timeit("kcmi.mi('Length', 'Diameter')", globals=globals(), number=30)/30


0.007938058799967014

In [14]:
import timeit
timeit.timeit("kcmi.pvalue('Length', 'Diameter')", globals=globals(), number=30)/600


0.008219302231664187

In [33]:
kcmi.pvalue('Length', 'Diameter', 'Height')

0.0

In [17]:
kcmi.pvalue('Length', 'Diameter', ['Height','Whole weight','Shucked weight'])

0.0

In [63]:
import timeit
timeit.timeit("kcmi.mi('Length', 'Diameter')", globals=globals(), number=20)/20


0.01945544370000789

In [ ]:
# sepsets = pbn.PC().compute_sepsets_of_size(pbn.PartiallyDirectedGraph().CompleteUndirected(
#             nodes=df.columns.to_list()),
#                                                    pbn.MutualInformation(
#                 df=df),
                                                   
#                                                    [],
#                                                    [],
#                                                    [],
#                                                    [],
#                                                    1)

In [ ]:
# import random
# pdag = pbn.PartiallyDirectedGraph().CompleteUndirected(
#     nodes=df.columns.to_list())

# rows_source = pd.DataFrame(itertools.combinations(df.columns, 2), columns=[
#                                    'Source', 'Target'])
# for idx, row in rows_source.iterrows():
#     if random.randint(0, 10) > 6:
#         pdag.remove_edge(row['Source'], row['Target'])

In [ ]:
# x = pbn.PC().compute_v_structures(pdag,
#     pbn.MutualInformation(
#     df=df),
#     alpha=0.05,
#     use_sepsets=False,
#     ambiguous_threshold = 0.5)
# x

In [ ]:
pdag.arcs()

NameError: name 'pdag' is not defined

In [ ]:
pdag.undirect('LungCancer', 'TuberculosisOrCancer')

In [ ]:
x = pbn.MeekRules.all_rules_sequential_interactive(pdag,
    )
x

[((4, 1), 1)]

In [ ]:
# pdag.arcs()

In [ ]:
# pdag.edges()

In [ ]:
# pdag.to_dag()

In [ ]:
# session_dict = [x for x in session_objects.values()][0]
# op_set = session_dict['op_set']
# score = session_dict['score']


# ghc = pbn.GreedyHillClimbing()
# session_dict['bn'] = ghc.estimate(
#     start=session_dict['bn'],
#     score=score,
#     operators=op_set,
#     arc_blacklist=[
#         list(d.values()) for d in session_dict['arc_blacklist']],
#     arc_whitelist=[
#         list(d.values()) for d in session_dict['arc_whitelist']],
#     max_indegree=session_dict['max_indegree'],
#     patience=50
# )

In [ ]:
# df = pd.read_csv('mushroom_mixed.csv', sep=None, engine='python')
# df = df.dropna(axis=1).iloc[:10000, :]
# df[df.select_dtypes(include='object').columns] = df.select_dtypes(
#     include='object').astype('category')
# df[df.select_dtypes(include='int').columns] = df.select_dtypes(
#     include='int').astype('str').astype('category')

In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = '2'



In [ ]:
app.run(debug=True, use_reloader=False, port=8050)

AssertionError: The setup method 'errorhandler' can no longer be called on the application. It has already handled its first request, any changes will not be applied consistently.
Make sure all imports, decorators, functions, etc. needed to set up the application are done before running it.

In [ ]:
# g = np.vectorize(lambda i, j: bic.local_score(bn, np.array(delta_frontend_df.columns.to_list())[i], [np.array(delta_frontend_df.columns.to_list())[j]]))
# pd.DataFrame(np.fromfunction(g, (len(df.columns), len(df.columns)), dtype=int), columns=df.columns)

In [ ]:
# def recv_process():
#     shutil.rmtree('prueba', ignore_errors=True)
#     os.mkdir("prueba")
#     pbn.hc(pd.DataFrame(np.ones(shape=(8,8))), bn_type=pbn.GaussianNetworkType(), score='bic', operators=['arcs'], patience=40)
    


In [ ]:
# from multiprocessing import Process
# p = Process(target=recv_process)
# p.start()
# import time
# import zmq

# context = zmq.Context()

# #  Socket to talk to server
# print("Connecting to hello world server…")
# socket = context.socket(zmq.REQ)
# socket.connect("tcp://localhost:5555")

# #  Do 10 requests, waiting each time for a response
# for request in range(10):
#     print("Sending request %s …" % request)
#     socket.send_string("Hello from Python!")

#     #  Get the reply.
#     me ssage = socket.recv_string()
#     print("Received reply %s [ %s ]" % (request, message))


# p.join()

In [ ]:
# jupyter_dash.inline_exceptions = False

: 